In [39]:
from gpiozero import Servo
from time import sleep

servo = Servo(17)

/usr/lib/python3/dist-packages/gpiozero/output_devices.py:1532: PWMSoftwareFallback: To reduce servo jitter, use the pigpio pin factory.See https://gpiozero.readthedocs.io/en/stable/api_output.html#servo for more info
  warnings.warn(PWMSoftwareFallback(


In [42]:
def scan(frequency=0.1, angle=100):
    try:
            for i in range(0,angle,1):
                servo.value = i*0.01
                sleep(frequency)
            for i in range(angle,-angle,-1):
                servo.value = i*0.01
                sleep(frequency)
            for i in range(-angle,0,1):
                servo.value = i*0.01
                sleep(frequency)
    except KeyboardInterrupt:
            print("Program stopped")

scan(frequency=0.05)

In [3]:
servo.value=0

In [34]:
import os
import sys
import time
import smbus
import numpy as np

from imusensor.MPU9250 import MPU9250

class NpanMPU9250(MPU9250.MPU9250):

    
    def calibrateAccelerometerNpan(self):
        """Caliberate Accelerometer by positioning it in 6 different positions
        
        This function expects the user to keep the imu in 6 different positions while caliberation. 
        It gives cues on when to change the position. It is expected that in all the 6 positions, 
        at least one axis of IMU is parallel to gravity of earth and no position is same. Hence we 
        get 6 positions namely -> +x, -x, +y, -y, +z, -z.
        """

        currentAccelRange = self.AccelRange
        currentFrequency = self.Frequency
        currentSRD = self.CurrentSRD
        self.setAccelRange("AccelRangeSelect2G")
        self.setLowPassFilterFrequency("AccelLowPassFilter20")
        self.setSRD(19)

        xbias = []
        ybias = []
        zbias = []
        xscale = []
        yscale = []
        zscale = []

        print ("Acceleration calibration is starting and keep placing the IMU in 6 different directions based on the instructions below")

        while True:
            input("Put the IMU in +Z")
            time.sleep(3)
            meanvals = self.__getAccelVals()
            print (meanvals)
            success = input("Success?:")
            if success == 'y':
                zscale, zbias = self.__assignBiasOrScale(meanvals[2], zscale, zbias)
                break
        while True:
            input("Put the IMU in -Z")
            time.sleep(3)
            meanvals = self.__getAccelVals()
            print (meanvals)
            success = input("Success?:")
            if success == 'y':
                zscale, zbias = self.__assignBiasOrScale(meanvals[2], zscale, zbias)
                break
        while True:
            input("Put the IMU in +X")
            time.sleep(3)
            meanvals = self.__getAccelVals()
            print (meanvals)
            success = input("Success?:")
            if success == 'y':
                xscale, xbias = self.__assignBiasOrScale(meanvals[0], xscale, xbias)
                break
        while True:
            input("Put the IMU in -X")
            time.sleep(3)
            meanvals = self.__getAccelVals()
            print (meanvals)
            success = input("Success?:")
            if success == 'y':
                xscale, xbias = self.__assignBiasOrScale(meanvals[0], xscale, xbias)
                break
        while True:
            input("Put the IMU in +Y")
            time.sleep(3)
            meanvals = self.__getAccelVals()
            print (meanvals)
            success = input("Success?:")
            if success == 'y':
                yscale, ybias = self.__assignBiasOrScale(meanvals[1], yscale, ybias)
                break
        while True:
            input("Put the IMU in -Y")
            time.sleep(3)
            meanvals = self.__getAccelVals()
            print (meanvals)
            success = input("Success?:")
            if success == 'y':
                yscale, ybias = self.__assignBiasOrScale(meanvals[1], yscale, ybias)
                break

        if len(xscale) != 2 or len(yscale) != 2 or len(zscale) != 2:
            print ("It looks like there were some external forces on sensor and couldn't get proper values. Please try again")
            return


        self.AccelBias[0] = -1*(xscale[0] + xscale[1])/(abs(xscale[0]) + abs(xscale[1]))
        self.AccelBias[1] = -1*(yscale[0] + yscale[1])/(abs(yscale[0]) + abs(yscale[1]))
        self.AccelBias[2] = -1*(zscale[0] + zscale[1])/(abs(zscale[0]) + abs(zscale[1]))

        self.AccelBias = -1*self.cfg.Gravity*self.AccelBias

        self.Accels[0] = (2.0*self.cfg.Gravity)/(abs(xscale[0]) + abs(xscale[1]))
        self.Accels[1] = (2.0*self.cfg.Gravity)/(abs(yscale[0]) + abs(yscale[1]))
        self.Accels[2] = (2.0*self.cfg.Gravity)/(abs(zscale[0]) + abs(zscale[1]))
        
        self.setAccelRange(currentAccelRange)
        self.setLowPassFilterFrequency(currentFrequency)
        self.setSRD(currentSRD)
    def __assignBiasOrScale(self, val, scale, bias):

        if val > 6.0 or val < -6.0 :
            scale.append(val)
        else:
            bias.append(val)
        return scale, bias
    def __getAccelVals(self):

            accelvals = np.zeros((100,3))
            for samples in range(1,100):
                self.readSensor()
                vals = self.AccelVals/self.Accels + self.AccelBias
                accelvals[samples] = vals
                time.sleep(0.02)
            meanvals = np.array([accelvals[:,0].mean(), accelvals[:,1].mean(), accelvals[:,2].mean()])
            return meanvals

In [35]:
address = 0x68
bus = smbus.SMBus(1)
imu = NpanMPU9250(bus, address)
imu.begin()

The name is wrong [113]


1

In [36]:
imu.calibrateAccelerometerNpan()

Acceleration calibration is starting and keep placing the IMU in 6 different directions based on the instructions below
Put the IMU in +Z
[ 6.80303367 -8.75523947 -7.71328968]
Success?:y
Put the IMU in -Z
[ 7.44067797 -5.0588527  11.5463205 ]
Success?:y
Put the IMU in +X
[  7.3344538  -16.59557196   4.26902518]
Success?:y
Put the IMU in -X
[ 7.30545854  2.38270519 -0.18764314]
Success?:y
Put the IMU in +Y
[16.75648846 -7.22376422  3.38978764]
Success?:y
Put the IMU in -Y
[-2.53937776 -6.50645479  2.71304404]
Success?:y


In [37]:
imu.saveCalibDataToFile("./calib_real_bolder.json")

In [10]:
imu.caliberateMagApprox()

In [4]:
imu.saveCalibDataToFile("./calib_real_bolder.json")

In [5]:
imu.loadCalibDataFromFile("./calib_real_bolder.json")

In [38]:
while True:
    imu.readSensor()
    print(imu.AccelVals,end="\r")
    time.sleep(0.1)

KeyboardInterrupt: 

In [12]:
imu.caliberateAccelerometer()
print ("Accel calibration Finisehd")
print (imu.AccelBias)
print (imu.Accels)

Acceleration calibration is starting and keep placing the IMU in 6 different directions based on the instructions below
Put the IMU in 1 position
[ 6.52596254 -8.53055015 -7.73496429]
[6.52596253788052]
[-8.530550150606548]
[-7.734964290226595]
Put the IMU in 2 position
[ 7.74257824 -6.56220661 11.76162406]
[6.52596253788052, 7.742578239414054]
[-8.530550150606548, -6.562206612039368]
[-7.734964290226595, 11.761624064087892]
Put the IMU in 3 position
[  6.61985594 -16.7858309    2.96150899]
[6.52596253788052, 7.742578239414054, 6.619855939574274]
[-8.530550150606548, -6.562206612039368, -16.785830897688257]
[-7.734964290226595, 11.761624064087892]
Put the IMU in 4 position
[7.58268531 2.49398737 0.57713974]
[6.52596253788052, 7.742578239414054, 6.619855939574274, 7.582685306477454]
[-8.530550150606548, -6.562206612039368, -16.785830897688257]
[-7.734964290226595, 11.761624064087892]
Put the IMU in 5 position
[-2.51445285 -6.26623229  2.78518501]
[6.52596253788052, 7.742578239414054, 6.